# Trying out sympy module

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from sympy.physics.quantum import Dagger
from Modules.sympy.solver import *

In [3]:
Spin = RDBasis("\\sigma", 'spin', dim=2)
sx = Spin.project(sp.Matrix([[0, 1], [1, 0]]))
sz = Spin.project(sp.Matrix([[1, 0], [0, -1]]))

a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("\\Omega_{z}", order = 0)
g = RDsymbol("g", order = 1)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz - hbar * g * (a + ad) * sx
H

\Omega_{z}*hbar*\sigma_{3}/2 - g*hbar*(a + {a^\dagger})*\sigma_{1} + hbar*omega*{a^\dagger}*a

In [6]:
# dictionary 
commutation = {
    a*ad: ad * a + 1,
}
group_by_infinite_operators(a*ad, commutation)

{1: 1, {a^\dagger}*a: 1}

# Better Solver

In [8]:
def generate_keys(order):
    keys = [] # will contain keys
    
    def deapth_first_search(current_key, current_sum):
        if current_sum > order: # if current_sum is bigger than order
            return # return nothing
        if current_key: # if current key is not empty
            keys.append(current_key[:]) # append current key to keys
        start = 0 if not current_key else 1 # if current key is empty, start the next number from 0, else from 1
        for i in range(start, order + 1): # looping over values between start and order
            current_key.append(i) # append current number to the key
            deapth_first_search(current_key, current_sum + i) # recurse down the tree
            current_key.pop() # backtrack

            
            
    deapth_first_search([], 0) # start search with inital sum to zero
    return keys

def custom_sort_key(sublist):
    """
    Custom sorting key function:
    - First, sorts by the sum of sublist elements in increasing order.
    - Second, sorts sublists with equal sums by moving sublists of length 2 containing a 0 in the first element to the bottom.
    """
    sublist_sum = sum(sublist)
    sublist_length = len(sublist)

    if sublist_length == 2 and sublist[0] == 0:
        # Ensure sublists of length 2 with a 0 in the first element go to the bottom
        return (sublist_sum, 1)
    else:
        return (sublist_sum, 0)

def rearrange_keys(list_of_sublists):
    # Sort sublists based on custom key
    sorted_sublists = sorted(list_of_sublists, key=custom_sort_key)

    return sorted_sublists


keys = generate_keys(3)

rearrange_keys(keys)

[[0],
 [1],
 [0, 1],
 [0, 1, 1],
 [1, 1],
 [2],
 [0, 2],
 [0, 1, 1, 1],
 [0, 1, 2],
 [0, 2, 1],
 [1, 1, 1],
 [1, 2],
 [2, 1],
 [3],
 [0, 3]]

In [4]:
Spin = RDBasis("\\sigma", 'spin', dim=2)
sx = Spin.project(sp.Matrix([[0, 1], [1, 0]]))
sz = Spin.project(sp.Matrix([[1, 0], [0, -1]]))

a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("\\Omega_{z}", order = 0)
g = RDsymbol("g", order = 1)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz - hbar * g * (a + ad) * sx
H

\Omega_{z}*hbar*\sigma_{3}/2 - g*hbar*(a + {a^\dagger})*\sigma_{1} + hbar*omega*{a^\dagger}*a

In [5]:
commutation_relations = {
    a*ad: ad * a + 1,
}

bosonic_basis = [[1, a, ad]]

In [11]:
sol = solver(H, bosonic_basis, Spin, order=3, full_diagonal=True, commutation_relations=commutation_relations)

|	0                               0                                1	|
|	1                               1                                1	|
|	2                              0-1                               2	|
|	3                             0-1-1                              3	|
|	4                              1-1                               2	|
|	5                               2                                1	|
|	6                              0-2                               2	|
|	7                            0-1-1-1                             4	|
|	8                             0-1-2                              3	|
|	9                             0-2-1                              3	|
|	10                            1-1-1                              3	|
|	11                             1-2                               2	|
|	12                             2-1                               2	|
|	13                              3                                1	|
Solvin

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
comm_relations = {
    a*ad : ad * a + 1,
}

sol[0].expand()#.subs(comm_relations).expand()

2*\Omega_{z}*g**2*hbar*{a^\dagger}*a*\sigma_{3}/(\Omega_{z}**2 - omega**2) + \Omega_{z}*g**2*hbar*\sigma_{3}/(\Omega_{z}**2 - omega**2) + \Omega_{z}*hbar*\sigma_{3}/2 + g**2*hbar*omega/(\Omega_{z}**2 - omega**2) + hbar*omega*{a^\dagger}*a

In [13]:
sol[1]['2'].expand()

\Omega_{z}*g**2*a**2*\sigma_{3}/(2*\Omega_{z}**2*omega - 2*omega**3) - \Omega_{z}*g**2*{a^\dagger}**2*\sigma_{3}/(2*\Omega_{z}**2*omega - 2*omega**3)

In [16]:
s0, sx, sy, sz = Spin._basis
sigma_p = sp.Rational(1,2) * (sx + sp.I * sy)
sigma_m = sp.Rational(1,2) * (sx - sp.I * sy)
S1 = (g / (omega + Omega_z) * sigma_p + g/(omega - Omega_z) * sigma_m) * ad - (g / (omega - Omega_z) * sigma_p + g/(omega + Omega_z) * sigma_m) * a
S1.simplify().expand()

I*\Omega_{z}*g*\sigma_{2}*a/(\Omega_{z}**2 - omega**2) + I*\Omega_{z}*g*\sigma_{2}*{a^\dagger}/(\Omega_{z}**2 - omega**2) + g*omega*\sigma_{1}*a/(\Omega_{z}**2 - omega**2) - g*omega*\sigma_{1}*{a^\dagger}/(\Omega_{z}**2 - omega**2)

In [17]:
S2 = Omega_z * g**2 / (2 * omega * (omega**2 - Omega_z**2)) * (ad * ad - a * a) * sz

S2.simplify().expand()

\Omega_{z}*g**2*a**2*\sigma_{3}/(2*\Omega_{z}**2*omega - 2*omega**3) - \Omega_{z}*g**2*{a^\dagger}**2*\sigma_{3}/(2*\Omega_{z}**2*omega - 2*omega**3)

# Comparison with PymaBlock

In [14]:
wr = RDsymbol("\\omega_{r}", order = 0)
wq = RDsymbol("\\omega_{q}", order = 0)
g = RDsymbol("g", order = 1)

Spin = RDBasis("\\sigma", 'spin', dim=2)
s0, sx, sy, sz = Spin._basis

sigma_p = sp.Rational(1,2)* (sx + sp.I * sy)
sigma_m = sp.Rational(1,2)* (sx - sp.I * sy)

a = RDBoson("a", subspace ="resonator")
ad = RDBoson("{a^\\dagger}", subspace ="resonator", is_annihilation=False)

H0 = wr * ad * a - wq * sp.Rational(1,2) * sz
H = H0 + g * a* sigma_m + g * ad * sigma_p
H

-\omega_{q}*\sigma_{3}/2 + \omega_{r}*{a^\dagger}*a + g*a*(\sigma_{1} - I*\sigma_{2})/2 + g*{a^\dagger}*(\sigma_{1} + I*\sigma_{2})/2

In [15]:
commutation_relations = {
    a*ad: ad * a + 1,
}

bosonic_basis = [[1, a, ad]]

In [16]:
H_perturbation_order = (solver(H, bosonic_basis, Spin, order=6, full_diagonal=False, commutation_relations=commutation_relations)[0]).expand()

|	0                               0                                1	|
|	1                               1                                1	|
|	2                              0-1                               2	|
|	3                             0-1-1                              3	|
|	4                              1-1                               2	|
|	5                               2                                1	|
|	6                              0-2                               2	|
|	7                            0-1-1-1                             4	|
|	8                             0-1-2                              3	|
|	9                             0-2-1                              3	|
|	10                            1-1-1                              3	|
|	11                             1-2                               2	|
|	12                             2-1                               2	|
|	13                              3                                1	|
|	14  

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<IPython.core.display.Math object>

<IPython.core.display.Math object>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
def subs_dict(expr, dictionary):
    expanded = expr.expand()
    while expanded != expanded.subs(dictionary):
        expanded = expanded.subs(dictionary).expand()
    return expanded

In [18]:
n = RDBoson('n', subspace='resonator', is_annihilation=False)
comm_relations = {
    a*ad : ad * a + 1,
    ad * a : n,
    a * n : n * a + a,
    ad * n : n * ad - ad,
}
result_mat = sp.zeros(2)
for k, val in group_by_finite_operators(subs_dict(H_perturbation_order, comm_relations).expand()).items():
    if k == 1:
        mat = sp.eye(2)
    else:
        mat = get_matrix(k)
    result_mat += val * mat

#result_mat = sp.simplify(result_mat).expand()
display(result_mat)
ordered_dict = group_by_order(result_mat[0, 0])
result_simp_dict = {}
for k, v in ordered_dict.items():
    result_simp_dict[k] = v.simplify().expand()
display_dict(result_simp_dict)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     -\omega_{q}/2 + \omega_{r}*n + 2*g**6/(36*\omega_{q}**5 - 180*\omega_{q}**4*\omega_{r} + 360*\omega_{q}**3*\omega_{r}**2 - 360*\omega_{q}**2*\omega_{r}**3 + 180*\omega_{q}*\omega_{r}**4 - 36*\omega_{r}**5) + 14*g**6*n/(36*\omega_{q}**5 - 180*\omega_{q}**4*\omega_{r} + 360*\omega_{

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Transmon Qubit coupled to resonator

In [19]:
at = RDBoson("{a_t}", subspace ="transmon")
adt = RDBoson("{a_t^\\dagger}", subspace ="transmon", is_annihilation=False)

ar = RDBoson("{a_r}", subspace ="resonator")
adr = RDBoson("{a_r^\\dagger}", subspace ="resonator", is_annihilation=False)

omega_t = RDsymbol("\\omega_{t}", order = 0)
omega_r = RDsymbol("\\omega_{r}", order = 0)
alpha = RDsymbol("\\alpha", order = 0)
g = RDsymbol("g", order = 1)

H0 =  -omega_t * ( adt * at  - sp.Rational(1,2) ) + sp.Rational(1,2) * alpha * (adt**2*at**2) + omega_r * (adr *ar + sp.Rational(1,2))
H = H0 - g * (adt  - at) * (adr - ar)
H

\alpha*{a_t^\dagger}**2*{a_t}**2/2 + \omega_{r}*(1/2 + {a_r^\dagger}*{a_r}) - \omega_{t}*(-1/2 + {a_t^\dagger}*{a_t}) - g*({a_t^\dagger} - {a_t})*({a_r^\dagger} - {a_r})

In [85]:
get_ansatz([[1, adt, at]], Spin, order=1)[0]

S^0_0*\sigma_{0} + S^1_0*{a_t^\dagger}*\sigma_{0} + S^2_0*{a_t}*\sigma_{0}

In [71]:
print(type(at**2*adt**2))
display_dict(group_by_diagonal(H))

<class 'sympy.core.mul.Mul'>


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [20]:
finite = RDBasis("1", 'finite', dim=1)
commutation = {
    at * adt : adt * at + 1,
    ar * adr : adr * ar + 1,
    ar * adt : adt * ar,
    adr * at : at * adr,
    ar * at : at * ar,
    adr * adt : adt * adr,
}

bosonic_basis = [[1, at, adt], [1, ar, adr]]
sol, S = solver(H, bosonic_basis, finite, order=3, full_diagonal=False, commutation_relations=commutation)

|	0                               0                                1	|
|	1                               1                                1	|
|	2                              0-1                               2	|
|	3                             0-1-1                              3	|
|	4                              1-1                               2	|
|	5                               2                                1	|
|	6                              0-2                               2	|
|	7                            0-1-1-1                             4	|
|	8                             0-1-2                              3	|
|	9                             0-2-1                              3	|
|	10                            1-1-1                              3	|
|	11                             1-2                               2	|
|	12                             2-1                               2	|
|	13                              3                                1	|
Solvin

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Solving S_2


<IPython.core.display.Math object>

<IPython.core.display.Math object>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
S['1']

g*{a_t^\dagger}*{a_r}*1_{0}/(\omega_{r} + \omega_{t}) - g*{a_t}*{a_r^\dagger}*1_{0}/(\omega_{r} + \omega_{t}) + g*{a_t^\dagger}*{a_r^\dagger}*1_{0}/(\omega_{r} - \omega_{t}) - g*{a_t}*{a_r}*1_{0}/(\omega_{r} - \omega_{t})

In [22]:

subs_to_zero = {
    at : 0,
    adt : 0,
    ar : 0,
    adr : 0,
}

(sol).expand()#.subs(subs_to_zero).expand().simplify().expand()

\alpha*{a_t^\dagger}**2*{a_t}**2/2 - 16*\omega_{r}**3*\omega_{t}*g**3*{a_t^\dagger}*{a_r^\dagger}/(3*\omega_{r}**6 - 9*\omega_{r}**4*\omega_{t}**2 + 9*\omega_{r}**2*\omega_{t}**4 - 3*\omega_{t}**6) + 16*\omega_{r}**3*\omega_{t}*g**3*{a_t^\dagger}*{a_r}/(3*\omega_{r}**6 - 9*\omega_{r}**4*\omega_{t}**2 + 9*\omega_{r}**2*\omega_{t}**4 - 3*\omega_{t}**6) + 16*\omega_{r}**3*\omega_{t}*g**3*{a_t}*{a_r^\dagger}/(3*\omega_{r}**6 - 9*\omega_{r}**4*\omega_{t}**2 + 9*\omega_{r}**2*\omega_{t}**4 - 3*\omega_{t}**6) - 16*\omega_{r}**3*\omega_{t}*g**3*{a_t}*{a_r}/(3*\omega_{r}**6 - 9*\omega_{r}**4*\omega_{t}**2 + 9*\omega_{r}**2*\omega_{t}**4 - 3*\omega_{t}**6) + 16*\omega_{r}*\omega_{t}**3*g**3*{a_t^\dagger}*{a_r^\dagger}/(3*\omega_{r}**6 - 9*\omega_{r}**4*\omega_{t}**2 + 9*\omega_{r}**2*\omega_{t}**4 - 3*\omega_{t}**6) - 16*\omega_{r}*\omega_{t}**3*g**3*{a_t^\dagger}*{a_r}/(3*\omega_{r}**6 - 9*\omega_{r}**4*\omega_{t}**2 + 9*\omega_{r}**2*\omega_{t}**4 - 3*\omega_{t}**6) - 16*\omega_{r}*\omega_{t}*

In [23]:
display_dict(group_by_infinite_operators((sol-H0) , commutation))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [29]:
Transmon = RDBasis("{a_t}", 'transmon', dim=3)
Resonator = RDBasis("{a_r}", 'resonator', dim=3)



a_mat = sp.Matrix(
    [
        [0, 1, 0],
        [0, 0, sp.sqrt(2)],
        [0, 0, 0]
    ]
)

ad_mat = sp.Matrix(
    [
        [0, 0, 0],
        [1, 0, 0],
        [0, sp.sqrt(2), 0]
    ]
)

at = Transmon.project(a_mat)
adt = Transmon.project(ad_mat)

ar = Resonator.project(a_mat)
adr = Resonator.project(ad_mat)


omega_t = RDsymbol("\\omega_{t}", order = 0)
omega_r = RDsymbol("\\omega_{r}", order = 0)
alpha = RDsymbol("\\alpha", order = 0)
g = RDsymbol("g", order = 1)

H0 =  -omega_t * ( adt * at  - sp.Rational(1,2) ) + sp.Rational(1,2) * alpha * (adt**2*at**2) + omega_r * (adr *ar + sp.Rational(1,2))
H = H0 - g * (adt  - at) * (adr - ar)
H

\alpha*({a_t}_{1}/3 - I*{a_t}_{2}/3 + sqrt(2)*{a_t}_{5}/3 - sqrt(2)*I*{a_t}_{6}/3)**2*({a_t}_{1}/3 + I*{a_t}_{2}/3 + sqrt(2)*{a_t}_{5}/3 + sqrt(2)*I*{a_t}_{6}/3)**2/2 + \omega_{r}*(1/2 + ({a_r}_{1}/3 - I*{a_r}_{2}/3 + sqrt(2)*{a_r}_{5}/3 - sqrt(2)*I*{a_r}_{6}/3)*({a_r}_{1}/3 + I*{a_r}_{2}/3 + sqrt(2)*{a_r}_{5}/3 + sqrt(2)*I*{a_r}_{6}/3)) - \omega_{t}*(-1/2 + ({a_t}_{1}/3 - I*{a_t}_{2}/3 + sqrt(2)*{a_t}_{5}/3 - sqrt(2)*I*{a_t}_{6}/3)*({a_t}_{1}/3 + I*{a_t}_{2}/3 + sqrt(2)*{a_t}_{5}/3 + sqrt(2)*I*{a_t}_{6}/3)) - g*(-({a_t}_{1}/3 + I*{a_t}_{2}/3 + sqrt(2)*{a_t}_{5}/3 + sqrt(2)*I*{a_t}_{6}/3) + {a_t}_{1}/3 - I*{a_t}_{2}/3 + sqrt(2)*{a_t}_{5}/3 - sqrt(2)*I*{a_t}_{6}/3)*(-({a_r}_{1}/3 + I*{a_r}_{2}/3 + sqrt(2)*{a_r}_{5}/3 + sqrt(2)*I*{a_r}_{6}/3) + {a_r}_{1}/3 - I*{a_r}_{2}/3 + sqrt(2)*{a_r}_{5}/3 - sqrt(2)*I*{a_r}_{6}/3)